# Практическое задание 2 (часть 2)

# Распознавание именованных сущностей из Twitter с помощью LSTM

## курс "Математические методы анализа текстов"


### ФИО: Алексеев Василий

## Введение

### Постановка задачи

В этом задании вы будете использовать рекуррентные нейронные сети для решения проблемы распознавания именованных сущностей (NER). Примерами именованных сущностей являются имена людей, названия организаций, адреса и т.д. В этом задании вы будете работать с данными twitter.

Например, вы хотите извлечь имена и названия организаций. Тогда для текста

    Yan Goodfellow works for Google Brain

модель должна извлечь следующую последовательность:

    B-PER I-PER    O     O   B-ORG  I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система введена, чтобы различать последовательные именованные сущности одного типа.

Решение этого задания будет основано на нейронных сетях, а именно на Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Numpy](http://www.numpy.org).
 
### Данные

Все данные содержатся в папке `./data`: `./data/train.txt`, `./data/validation.txt`, `./data/test.txt`.

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1s1rFOFMZTBqtJuQDcIvW-8djA78iUDcx)

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [22]:
from functools import reduce

## Часть 1. Подготовка данных (2 балла)

### Загрузка данных

Мы будем работать с данными, которые содержат твиты с тегами именованных сущностей. Каждая строка файла содержит пару токен(слово или пунктуация) и тег, разделенные пробелом. Различные твиты разделены пустой строкой.

Функция *read_data* считывает корпус из *file_path* и возвращает два списка: один с токенами и один с соответствующими токенам тегами. Также она заменяет все ники (токены, которые начинаются на символ *@*) на токен `<USR>` и url-ы (токены, которые начинаются на *http://* или *https://*) на токен `<URL>`. 

Вам необходимо реализовать эту функцию.

In [2]:
USR = '<USR>'
URL = '<URL>'

In [7]:
def read_data(file_path):
    sents_tokens = []
    sents_tags = []

    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    lines = open(file_path, 'r').readlines()
    
    tokens = []
    tags = []
    
    for l in lines:
        if len(l.strip()) == 0:
            sents_tokens.append(tokens)
            sents_tags.append(tags)
            
            tokens = []
            tags = []
            
            continue
        
        token, tag = l.strip().split()
        
        if token.startswith('@'):
            token = USR
        
        elif token.startswith('http://') or token.startswith('https://'):
            token = URL
        
        tokens.append(token)
        tags.append(tag)
    
    return sents_tokens, sents_tags

Теперь мы можем загрузить 3 части данных:
 - *train* для тренировки модели;
 - *validation* для валидации и подбора гиперпараметров;
 - *test* для финального тестирования.

In [8]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

Всегда полезно знать, с какими данными вы работаете. Выведем небольшую часть.

In [10]:
train_tokens[1]

['Apple',
 'MacBook',
 'Pro',
 'A1278',
 '13.3',
 '"',
 'Laptop',
 '-',
 'MD101LL/A',
 '(',
 'June',
 ',',
 '2012',
 ')',
 '-',
 'Full',
 'read',
 'by',
 'eBay',
 '<URL>',
 '<URL>']

In [564]:
train_tags[1]

['B-product',
 'I-product',
 'I-product',
 'I-product',
 'I-product',
 'I-product',
 'I-product',
 'I-product',
 'I-product',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-company',
 'O',
 'O']

In [11]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения.

- {token}$\to${token id}: устанавливает соответствие между токеном и строкой в embedding матрице;
- {tag}$\to${tag id}: one hot encoding тегов.


Теперь вам необходимо реализовать функцию *build_dict*, которая должна возвращать словарь {token or tag}$\to${index} и контейнер, задающий обратное отображение.

In [12]:
from collections import defaultdict

In [261]:
def build_dict(tokens_or_tags, special_tokens):
    """
    tokens_or_tags: a list of lists of tokens or tags
    special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens to indices and vice versa
    # Add special tokens to dictionaries
    # The first special token must have index 0

    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    index = 0
    
    def get_index():
        nonlocal index
        
        i = index
        
        index += 1
        
        return i
    
    for t in special_tokens:
        tok2idx[t] = get_index()

    for s in tokens_or_tags:
        for t in s:
            if t in tok2idx:
                continue
            
            tok2idx[t] = get_index()
    
    idx2tok = {i: tok for tok, i in tok2idx.items()}
    
    return tok2idx, idx2tok

После реализации функции *build_dict* вы можете создать словари для токенов и тегов. В нашем случае специальными токенами будут:
 - `<UNK>` токен для обозначаения слов, которых нет в словаре;
 - `<PAD>` токен для дополнения предложений одного батча до одинаковой длины.

In [563]:
train_tokens[4]

['<USR>',
 '<USR>',
 '<USR>',
 '<USR>',
 '<USR>',
 'still',
 'perception',
 ',',
 'and',
 'what',
 'you',
 'key',
 'off',
 'may',
 'differ',
 'from',
 'me',
 '.',
 'Back',
 'to',
 'Selma',
 'example',
 ',',
 'iow',
 '.']

In [562]:
train_tag_idxs[4]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0]

In [565]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O', 'PAD']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [566]:
list(token2idx.items())[:10]

[('<UNK>', 0),
 ('<PAD>', 1),
 ('RT', 2),
 ('<USR>', 3),
 (':', 4),
 ('Online', 5),
 ('ticket', 6),
 ('sales', 7),
 ('for', 8),
 ('Ghostland', 9)]

In [567]:
list(idx2token.items())[:10]

[(0, '<UNK>'),
 (1, '<PAD>'),
 (2, 'RT'),
 (3, '<USR>'),
 (4, ':'),
 (5, 'Online'),
 (6, 'ticket'),
 (7, 'sales'),
 (8, 'for'),
 (9, 'Ghostland')]

In [568]:
list(tag2idx.items())[:10]

[('O', 0),
 ('PAD', 1),
 ('B-musicartist', 2),
 ('I-musicartist', 3),
 ('B-product', 4),
 ('I-product', 5),
 ('B-company', 6),
 ('B-person', 7),
 ('B-other', 8),
 ('I-other', 9)]

### Генератор батчей

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины. Для некоторых фреймворков (таких как tensorflow) это необходимо сделать до подачи батча в нейронную сеть. В случае с pytorch это можно сделать как вне архитектуры нейронной сети, так и внутри. Мы выбрали более универсальный вариант и наш генератор батчей дополняет все последовательности внутри одного батча до одной длины.

Генератор батчей разбивает последовательность входных предложений и тегов на батчи размера batch_size. Размер последнего батча может быть меньше, если allow_smaller_last_batch is True, иначе последний батч исключается из генератора. Если включён параметр shuffle, данные перед разделением на батчи будут перемешаны. 

In [577]:
def batches_generator(batch_size, tokens_idxs, tags_idxs,
                      shuffle=True, allow_smaller_last_batch=True, device='cpu'):
    """
    Generates padded batches of tags_idxs and tags_idxs.
    
    batch_size : int, number of objects in one batch
    tokens_idxs : list of list of int
    tags_idxs : list of list of int
    shuffle : bool
    allow_smaller_last_batch : bool
    device: str, cpu or cuda:x
    
    yield x, y: torch.LongTensor and torch.LongTensor
    x - batch of tokens_idxs, y - batch of tags_idxs
    """
    n_samples = len(tokens_idxs)
    
    # Shuffle data if shuffle is True.
    # Don't modify the original tokens and tags!
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    if shuffle:
        indices = np.arange(0, len(tokens_idxs))
        
        np.random.shuffle(indices)
        
        tokens_idxs = np.array(tokens_idxs)[indices].tolist()
        tags_idxs = np.array(tags_idxs)[indices].tolist()
    
    # Get the number of batches
    # n_batches =  ### YOUR CODE HERE ###
    
    n_batches = int(np.ceil(
       len(tokens_idxs) / batch_size
    ))
    
    # For each k yield pair x and y
    for k in range(n_batches):
        
        ######################################
        ######### YOUR CODE HERE #############
        ######################################

        x = tokens_idxs[k * batch_size : k * batch_size + batch_size]
        y =   tags_idxs[k * batch_size : k * batch_size + batch_size]
        
        if k == n_batches - 1 and len(x) < batch_size and not allow_smaller_last_batch:
            return
        
        length = max(len(s) for s in x)
        
        for i, s in enumerate(x):
            if len(s) < length:
                x[i] = s + [token2idx['<PAD>']] * (length - len(s))
                y[i] = y[i] + [tag2idx['PAD']] * (length - len(s))
        
        x = torch.LongTensor(x)
        y = torch.LongTensor(y)
        
        x = x.long().to(device)
        y = y.long().to(device)
        
        yield x, y

Протестируйте ваш генератор батчей:

In [578]:
def get_token_idxs(tokens):
    return [
        [token2idx[t] for t in s]
        for s in tokens
    ]

def get_tag_idxs(tags):
    return [
        [tag2idx[t] for t in s]
        for s in tags
    ] 

In [579]:
train_idxs = get_token_idxs(train_tokens)
train_tag_idxs = get_tag_idxs(train_tags)

In [580]:
test_idxs = get_token_idxs(test_tokens)
test_tag_idxs = get_tag_idxs(test_tags)

In [581]:
val_idxs = get_token_idxs(validation_tokens)
val_tag_idxs = get_tag_idxs(validation_tags)

In [582]:
len(train_idxs), len(test_idxs), len(val_idxs)

(5795, 724, 724)

In [588]:
tag2idx['PAD']

1

In [590]:
test_nonrandom_batch_generator = batches_generator(
    batch_size=3,
    tokens_idxs=train_idxs[:7],
    tags_idxs=train_tag_idxs[:7],
    shuffle=False,
    allow_smaller_last_batch=True
)

batch_lengths = [3, 3, 1]
sequence_lengths = [26, 25, 8]
some_pad_tensor = torch.LongTensor([token2idx['<PAD>']] * 12)
some_outside_tensor = torch.LongTensor([tag2idx['PAD']] * 12)

for i, (tokens_batch, tags_batch) in enumerate(test_nonrandom_batch_generator):
    assert tokens_batch.dtype == torch.int64, 'tokens_batch is not LongTensor'
    assert tags_batch.dtype == torch.int64, 'tags_batch is not LongTensor'
    
    assert len(tokens_batch) == batch_lengths[i], 'wrong batch length'
    
    for one_token_sequence in tokens_batch:
        assert len(one_token_sequence) == sequence_lengths[i], 'wrong length of sequence in batch'
    
    if i == 0:
        assert torch.all(tokens_batch[2][-12:] == some_pad_tensor), "wrong padding"
        assert torch.all(tags_batch[2][-12:] == some_outside_tensor), "wrong O tag"

## Часть 2. BiLSTM (3 балла)

Определите архитектуру сети, используя библиотеку pytorch. 

**Замечания:**
1. Для улучшения качества сети предлагается использовать дополнительный Embedding слой на входе (каждому слову ставится в соответствие обучаемый вектор). 

2. Не забудьте, что `<PAD>` токены не должны учавствовать в подсчёте функции потерь.

In [725]:
# https://cs230-stanford.github.io/pytorch-nlp.html

def custom_cross_entropy_loss(outputs, labels):
    # reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)  

    # mask out 'PAD' tokens
    mask1 = (labels != tag2idx['PAD']).float()
    mask2 = (labels != tag2idx['O']).float()
    
#     print(labels)
#     print(mask1)
#     print(mask2)
#     print(mask1 * mask2)
#     print(int(torch.sum(mask1 * mask2)))

    # the number of tokens is the sum of elements in mask
    num_tokens = int(torch.sum(mask1 * mask2))

    # pick the values corresponding to labels and multiply by mask
    outputs = outputs[range(outputs.shape[0]), labels] * mask1 * mask2

    # cross entropy loss for all non 'PAD' tokens
    return -torch.sum(outputs) / max(num_tokens, 1)

In [756]:
class BiLSTMModel(torch.nn.Module):
    def __init__(self,
                 vocabulary_size,
                 n_tags,
                 PAD_index,
                 embedding_dim,
                 rnn_hidden_size,
                 dropout_zeroed_probability,
                 device='cpu'):
        '''
        Defines neural network structure.
        
        architecture: input -> Embedding -> BiLSTM -> Dropout -> Linear
        optimizer: Adam
        
        ----------
        Parameters
        
        vocabulary_size: int, number of words in vocabulary.
        n_tags: int, number of tags.
        PAD_index: int, index of padding character. Used for loss masking.
        embedding_dim: int, dimension of words' embeddings.
        rnn_hidden_size: int, number of hidden units in each LSTM cell
        dropout_zeroed_probability: float, dropout zeroed probability for Dropout layer.
        device: str, cpu or cuda:x
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        super().__init__()
        
        self.vocabulary_size = vocabulary_size
        self.n_tags = n_tags
        self.PAD_index = PAD_index
        self.embedding_dim = embedding_dim
        self.rnn_hidden_size = rnn_hidden_size
        self.dropout_zeroed_probability = dropout_zeroed_probability
        self.device = device

        self.embedding = nn.Embedding(
            self.vocabulary_size,
            self.embedding_dim,
            padding_idx=self.PAD_index
        )

        self.lstm = nn.LSTM(
            input_size=self.embedding.embedding_dim,
            hidden_size=rnn_hidden_size // 2,
            bidirectional=True
        )
        
        self.dropout = nn.Dropout(self.dropout_zeroed_probability)
        
        self.linear = nn.Linear(2 * self.lstm.hidden_size, len(tag2idx))
        
    def forward(self, x_batch):
        '''
        Makes forward pass.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        
        # input -> Embedding -> BiLSTM -> Dropout -> Linear
        
        x_batch = torch.LongTensor(x_batch).to(self.device)

        embeddings = self.embedding(x_batch)
        lstm_output, _ = self.lstm(embeddings)
        dropout_output = self.dropout(lstm_output)
        pred_scores = self.linear(dropout_output)

        pred_probs = torch.nn.functional.log_softmax(pred_scores, dim=2)

#         print(x_batch.shape)
#         print('embeddings', embeddings.shape)
#         print('lstm_output', lstm_output.shape)
#         print('dropout_output', dropout_output.shape)
#         print('pred_scores', pred_scores.shape)
#         print('pred_probs', pred_probs.shape)
#         print('pred_tags', pred_tags.shape)
        
#         for s in x_batch:
#             pred_sent = []

#             for t in s:
        
#                 embedding = self.embedding(t)
#                 print(embedding)
#                 lstm_output, _ = self.lstm(embedding)
#                 dropout_output = self.dropout(lstm_output)
#                 pred_scores = self.linear(dropout_output)
                
#                 pred_tag = torch.nn.functional.log_softmax(pred_scores)
                
#                 print(x_batch.shape)
#                 print(t.shape)
#                 print('embedding', embedding.shape)
#                 print('lstm_output', lstm_output.shape)
#                 print('dropout_output', dropout_output.shape)
#                 print('pred_scores', pred_scores.shape)
#                 break
                
#                 pred_sent.append(pred_tag)
            
#             pred_batch.append(pred_sent)
        
#         print(pred_batch)
        
        return pred_probs.to(device=self.device)
        
    def predict_for_batch(self, x_batch):
        '''
        Returns predictions for x_batch.
        
        return type: torch.LongTensor
        return shape: (number of samples in batch, number words in sentence.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ###################################### 
        
        pred_probs = self.forward(x_batch)
        pred_tags = torch.argmax(pred_probs, dim=2)
        
        return pred_tags.tolist()
    
    def train_on_batch(
        self,
        x_batch, y_batch,
        optimizer,
        criterion,
        loss_function=custom_cross_entropy_loss,
        lr=1e-4
    ):
        '''
        Trains model on the given batch.
        
        ----------
        Parameters
        x_batch: np.ndarray with shape (number of samples in batch, number words in sentence).
        y_batch: np.ndarray with shape (number of samples in batch).
        optimizer: torch.optimizer class
        loss_function: torch loss class
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        
        x_batch = x_batch.tolist()
        y_batch = y_batch.tolist()
        
        optimizer.zero_grad()

#         criterion = loss_class(ignore_index=tag2idx['PAD'])  # tag2idx['O']
#         optimizer = optimizer_class(self.parameters(), lr=lr)
        
        pred_batch = self.forward(x_batch)

        for sent_tokens, sent_preds, sent_tags in zip(x_batch, pred_batch, y_batch):
#             pad_indices = [i for i, tok_idx in enumerate(sent_tokens)
#                            if tok_idx == token2idx['<PAD>']]

            if loss_function is None:
                loss = criterion(
                    torch.FloatTensor(sent_preds),
                    torch.LongTensor(sent_tags)
                )
            else:
                loss = loss_function(
                    torch.FloatTensor(sent_preds),
                    torch.LongTensor(sent_tags)
                )

            loss.backward(retain_graph=True)

            optimizer.step()

Для тестирования сети мы подготовили для вас две функции:
 - *predict_tags*: получает батч данных и трансформирует его в список из токенов и предсказанных тегов;
 - *eval_conll*: вычисляет метрики precision, recall и F1

In [727]:
from evaluation_ner import precision_recall_f1

In [728]:
def predict_tags(model, token_idxs_batch):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(token_idxs_batch)
    
    tags_batch, tokens_batch = [], []
    
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            if token_idx != token2idx['<PAD>']:
                tags.append(idx2tag[tag_idx])
                tokens.append(idx2token[token_idx])
        
        tags_batch.append(tags)
        tokens_batch.append(tokens)

    return tags_batch, tokens_batch
    
    
def eval_conll(model, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    
    for x_batch, y_batch in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, x_batch.tolist())
        ground_truth_tags = [idx2tag[tag_idx] for tag_idx in y_batch.tolist()[0]]

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['PAD'])
        y_pred.extend(tags_batch[0] + ['PAD'])
    
    results = precision_recall_f1(
        y_true, y_pred,
        print_results=True,
        short_report=short_report
    )
    
    return results

In [729]:
model = BiLSTMModel(
    vocabulary_size=len(token2idx),
    n_tags=len(tag2idx),
    PAD_index=token2idx['<PAD>'],
    embedding_dim=100,
    rnn_hidden_size=200,
    dropout_zeroed_probability=0.4
)

In [730]:
batch_size = 3

In [731]:
num_batches = len([s for s in batches_generator(batch_size, train_idxs, train_tag_idxs)])

In [732]:
tag2idx['PAD']

1

In [733]:
train_tag_idxs[0]

[0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [734]:
log_after = 200

for i, (x_batch, y_batch) in enumerate(batches_generator(
        batch_size, train_idxs, train_tag_idxs)):

    if i % log_after == 0:
        print(f'{i} / {num_batches}')
    
    model.train_on_batch(x_batch, y_batch)

0 / 1932
200 / 1932
400 / 1932
600 / 1932
800 / 1932
1000 / 1932
1200 / 1932
1400 / 1932
1600 / 1932
1800 / 1932


In [735]:
predict_tags(model, train_idxs[10:11])

([['B-geo-loc',
   'I-other',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-other',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-geo-loc',
   'B-person',
   'B-geo-loc',
   'B-geo-loc']],
 [['Big',
   'Day',
   'tomorrow',
   '-',
   'Thanks',
   'to',
   'all',
   'of',
   'Team',
   'Pioneers',
   'who',
   'have',
   'been',
   'busy',
   'unwrapping',
   'tables',
   'and',
   'unpacking',
   'boxes',
   'to',
   'ensure',
   'school',
   'is',
   'ready',
   '!']])

In [487]:
2 * (1 / 100) * (22 / 100) / (1 / 100 + 22 / 100)

0.019130434782608695

In [736]:
eval_conll(model, train_idxs, train_tag_idxs)

processed 105778 tokens with 4489 phrases; found: 99943 phrases; correct: 934.

precision:  0.93%; recall:  20.81%; F1:  1.79



OrderedDict([('D',
              OrderedDict([('precision', 0),
                           ('recall', 0),
                           ('f1', 0),
                           ('n_predicted_entities', 0),
                           ('n_true_entities', 0)])),
             ('company',
              OrderedDict([('precision', 3.836667580158948),
                           ('recall', 21.77293934681182),
                           ('f1', 6.5237651444548),
                           ('n_predicted_entities', 3649),
                           ('n_true_entities', 643)])),
             ('facility',
              OrderedDict([('precision', 0.0),
                           ('recall', 0.0),
                           ('f1', 0),
                           ('n_predicted_entities', 21),
                           ('n_true_entities', 314)])),
             ('geo-loc',
              OrderedDict([('precision', 0.8574700080672444),
                           ('recall', 66.16465863453816),
                      

### Эксперименты

Задайте BiLSTMModel. Рекомендуемые параметры:
- *batch_size*: 32;
- начальное значение *learning_rate*: 0.01-0.001
- *dropout_zeroed_probability*: 0.7-0.9
- *embedding_dim*: 100-200
- *rnn_hidden_size*: 150-200

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1 мере на валидационной и тестовой выборках было не меньше 0.35. 

Если сеть плохо обучается, попробуйте использовать следующие модификации:
    * используйте gradient clipping
    * на каждой итерации уменьшайте learning rate (например, в 1.1 раз)
    * попробуйте вместо Adam другие оптимизаторы 
    * экспериментируйте с dropout

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################        

In [757]:
model = BiLSTMModel(
    vocabulary_size=len(token2idx),
    n_tags=len(tag2idx),
    PAD_index=token2idx['<PAD>'],
    embedding_dim=100,
    rnn_hidden_size=150,
    dropout_zeroed_probability=0.7
)

In [758]:
batch_size = 32
lr = 0.01

In [759]:
num_batches = len([s for s in batches_generator(batch_size, train_idxs, train_tag_idxs)])

In [760]:
num_batches

182

In [ ]:
num_epochs = 10

criterion = torch.nn.CrossEntropyLoss(ignore_index=tag2idx['PAD'])  # tag2idx['O']
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for e in range(num_epochs):
    print('epoch #', e)
    
    log_after = 100

    for i, (x_batch, y_batch) in enumerate(batches_generator(
            batch_size, train_idxs, train_tag_idxs)):

        if i % log_after == 0:
            print(f'{i} / {num_batches}')

        model.train_on_batch(
            x_batch,
            y_batch,
            optimizer=optimizer,
            criterion=criterion,
            loss_function=custom_cross_entropy_loss
        )

epoch # 0
0 / 182
100 / 182
epoch # 1
0 / 182


In [532]:
eval_conll(model, train_idxs, train_tag_idxs)

processed 105778 tokens with 4489 phrases; found: 99547 phrases; correct: 773.

precision:  0.78%; recall:  17.22%; F1:  1.49



OrderedDict([('company',
              OrderedDict([('precision', 0.9076682316118936),
                           ('recall', 4.5101088646967336),
                           ('f1', 1.5112037519541428),
                           ('n_predicted_entities', 3195),
                           ('n_true_entities', 643)])),
             ('facility',
              OrderedDict([('precision', 0.0),
                           ('recall', 0.0),
                           ('f1', 0),
                           ('n_predicted_entities', 500),
                           ('n_true_entities', 314)])),
             ('geo-loc',
              OrderedDict([('precision', 0.8508674521567327),
                           ('recall', 59.23694779116466),
                           ('f1', 1.6776376587002573),
                           ('n_predicted_entities', 69341),
                           ('n_true_entities', 996)])),
             ('movie',
              OrderedDict([('precision', 0.0),
                           ('

In [533]:
eval_conll(model, test_idxs, test_tag_idxs)

processed 13258 tokens with 604 phrases; found: 12482 phrases; correct: 117.

precision:  0.94%; recall:  19.37%; F1:  1.79



OrderedDict([('company',
              OrderedDict([('precision', 1.2658227848101267),
                           ('recall', 5.952380952380952),
                           ('f1', 2.0876826722338206),
                           ('n_predicted_entities', 395),
                           ('n_true_entities', 84)])),
             ('facility',
              OrderedDict([('precision', 0.0),
                           ('recall', 0.0),
                           ('f1', 0),
                           ('n_predicted_entities', 43),
                           ('n_true_entities', 47)])),
             ('geo-loc',
              OrderedDict([('precision', 1.0761305094447624),
                           ('recall', 56.96969696969697),
                           ('f1', 2.112359550561798),
                           ('n_predicted_entities', 8735),
                           ('n_true_entities', 165)])),
             ('movie',
              OrderedDict([('precision', 0.0),
                           ('recall'

In [508]:
eval_conll(model, val_idxs, val_tag_idxs)

processed 12836 tokens with 537 phrases; found: 12040 phrases; correct: 70.

precision:  0.58%; recall:  13.04%; F1:  1.11



OrderedDict([('company',
              OrderedDict([('precision', 0.7716049382716049),
                           ('recall', 4.807692307692308),
                           ('f1', 1.3297872340425532),
                           ('n_predicted_entities', 648),
                           ('n_true_entities', 104)])),
             ('facility',
              OrderedDict([('precision', 0.0),
                           ('recall', 0.0),
                           ('f1', 0),
                           ('n_predicted_entities', 146),
                           ('n_true_entities', 34)])),
             ('geo-loc',
              OrderedDict([('precision', 0.8202653799758746),
                           ('recall', 30.08849557522124),
                           ('f1', 1.5969938938468764),
                           ('n_predicted_entities', 4145),
                           ('n_true_entities', 113)])),
             ('movie',
              OrderedDict([('precision', 0.0),
                           ('reca

## Бонусная часть. Улучшение качества теггера (4 балла).

Улучшите качество теггера на данной задаче.

Бонусные баллы будут начисляться в зависимости от результата f1-меры (одновременно на тестовой и валидационной выборках!).

+ 1 балл — $> 0.38$
+ 2 балла — $> 0.4$
+ 3 балла — $> 0.425$
+ 4 балла — $> 0.45$

Разрешается использовать любые разумные способы (в том числе и не рассматривающиеся в курсе). Под неразумными способами понимаются любые, в которых используются модели, обученные на dev или test, а также модели, использующие утечки в данных, не относяющиеся к смыслу задачи.

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################        